In [ ]:
from PC6_encoding import get_PC6_features_labels
from model import train_pc6_model
from model_tools import learning_curve, evalution_metrics
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
import joblib

In [ ]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [ ]:
# load train data
train_data, train_labels = get_PC6_features_labels('../data/pos_train_ds2.fasta',
                                                      '../data/neg_train_ds2.fasta',length=50)

test_data, test_labels = get_PC6_features_labels('../data/pos_test_ds2.fasta',
                                                      '../data/neg_test_ds2.fasta',length=50)

## CNN model training

In [ ]:
train_data_, test_data_, train_labels_, test_labels_ = train_test_split(train_data, train_labels, test_size= 0.1, random_state = 1, stratify = train_labels)


In [ ]:
t_m = train_pc6_model(train_data_, train_labels_, test_data_, test_labels_, 'pc6_DS2', path = '../PC6_model')

In [ ]:
model = load_model('../PC6_model/pc6_DS2_final_weights.h5')
labels_score = model.predict(test_data)
evalution_metrics(test_labels, labels_score)

## Random forest

In [ ]:
# reshape
train_data = train_data.reshape(train_data.shape[0],-1)
test_data = test_data.reshape(test_data.shape[0],-1)

In [ ]:
from sklearn import ensemble
forest = ensemble.RandomForestClassifier(n_estimators = 100)
forest_fit = forest.fit(train_data, train_labels)
joblib.dump(forest, '../PC6_model/forest_pc6_DS2.pkl')

In [ ]:
forest = joblib.load('../PC6_model/forest_pc6_DS2.pkl')
labels_score = forest.predict(test_data)
evalution_metrics(test_labels, labels_score)

## Support vector machine

In [ ]:
from sklearn import svm
svc = svm.SVC()
svc_fit = svc.fit(train_data, train_labels)
joblib.dump(svc, '../PC6_model/svm_pc6_DS2.pkl')

In [ ]:
svc = joblib.load('../PC6_model/svm_pc6_DS2.pkl')
labels_score = svc.predict(test_data)
evalution_metrics(test_labels, labels_score)